Web Scraping by using BeautifulSoup

In [ ]:
import pandas as pd
import bs4
import requests
import openpyxl
import os

In [ ]:
data = requests.get('https://www.thejewelrymagazine.com/?s=Sustainable+material+')

In [ ]:
soup = bs4.BeautifulSoup(data.text,'html.parser')
soup

In [ ]:
title = soup.find('div',{'class':'td_module_16 td_module_wrap td-animation-stack td-meta-info-hide'})

In [ ]:
title.find('h3',{'class':'entry-title td-module-title'}).text

In [ ]:
tag = title.find('div',{'class':'td-module-meta-info'})

In [ ]:
cats = title.find('div',{'class':'td-module-meta-info'})

In [ ]:
cats.find('a').text

In [ ]:
desc = title.find('div',{'class':'td-excerpt'}).text.replace('\r','').replace('\n','')

In [ ]:
len(soup.find('div',{'class':'td_module_16 td_module_wrap td-animation-stack td-meta-info-hide'}))

In [ ]:
i_page = 1
title_list = []
cats_list = []
desc_list = []
while i_page <= 5:
  data = requests.get(f'https://www.thejewelrymagazine.com/page/{i_page}/?s=Sustainable+material+')
  soup = bs4.BeautifulSoup(data.text)
  for c in soup.find_all('div', class_='td_module_16 td_module_wrap td-animation-stack td-meta-info-hide'):
    title_list.append(c.find('h3', class_= 'entry-title td-module-title').find('a').text.strip())
    cats_list.append(c.find('div', class_='td-module-meta-info').text.strip())
    desc_list.append(c.find('div', class_='td-excerpt').text.strip())
  print('Complete page number: ',i_page)
  i_page += 1

In [ ]:
table = pd.DataFrame([title_list,cats_list,desc_list]).transpose()
table.columns = ['title','catagory','detail']

In [ ]:
df = pd.DataFrame(table)
df

In [ ]:
df.info()

In [ ]:
df.duplicated()

In [ ]:
df.drop_duplicates(inplace=True)
df

In [ ]:
df2 = df.drop([13,21])
df2

In [ ]:
df3 = df2.replace({'\® ':'','\~':'','è':'e','’':'','‘':"","–":'-','—':'-','é':'e','\|':'','\$':''}, regex = True)
df3

In [ ]:
df3.to_csv(r'C:\Users\acer\Desktop\newsinjewelrymagazine.csv',index = False )

Uploaded csv file to DataPlatform and Covert file to parquet

In [ ]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
dir_raw_id = 45
dir_process_id = 123

In [ ]:
datanode = DataNode()

In [ ]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='newsinjewelrymagazine.csv'))
meta

In [ ]:
df = pd.read_csv(fp)
df

In [ ]:
datanode.write(df=df, directory=dir_process_id, name="sustainable_material_in_jewelrymagazine", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='newsinjewelrymagazine.csv')])